In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_quote(soup):
    # locate target elements from the soup
    quote_text = soup.find('span')
    author = soup.find('small')
    author_url = soup.find('a')
    tags = soup.find_all('a', {'class': 'tag'})

    # extract information from the retrieved elements
    quote_text = quote_text.text
    author = author.text
    author_url = author_url.get('href')
    tags_ls = []
    for tag in tags:
        tag = tag.text
        tags_ls.append(tag)  
    
    # sort the info retrieved
    author_url = 'http://quotes.toscrape.com' + author_url
    tags = ';'.join(tags_ls)
    
    # store the results
    results_dt = {
    'author': author,
    'author_url': author_url,
    'tags': ';'.join(tags_ls), 
    'quote_text': quote_text
    }
    
    return results_dt

In [ ]:
def get_quotes(page_number):
    # define the URL
    base_url = 'http://quotes.toscrape.com/page/'
    page_number = str(page_number)
    url = base_url + page_number
    
    # make a request to retrieve HTML codes and make the soup
    r = requests.get(url)
    c = r.content
    soup = BeautifulSoup(c, 'lxml')
    
    # locate all the quotes
    quotes = soup.find_all('div', {'class': 'quote'})
    
    # parse each quote using for loops
    outputs = []
    for quote in quotes:
        quote = get_quote(quote)
        outputs.append(quote)
        
    # return the outputs
    return outputs

In [ ]:
def scrape_quotes(start, end):
    outputs = []
    for i in list(range(start, end + 1)):
        outputs += get_quotes(i)
    outputs = pd.DataFrame(outputs)
    return outputs

In [ ]:
outputs = scrape_quotes(1, 10)

In [ ]:
outputs.head()

In [ ]:
outputs.to_csv('quotes_extracted.csv')